In [ ]:
{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Install required packages\n",
    "!pip install --upgrade transformers torch peft huggingface-hub\n",
    "\n",
    "# Log in to Hugging Face (run this cell and paste your token when prompted)\n",
    "!huggingface-cli login\n",
    "\n",
    "# Import libraries\n",
    "import os\n",
    "import torch\n",
    "from transformers import AutoTokenizer, AutoModelForCausalLM\n",
    "from peft import PeftModel, LoraConfig\n",
    "from datasets import load_dataset\n",
    "\n",
    "# Define cache directory\n",
    "cache_dir = \"./pretrained-models\"\n",
    "os.makedirs(cache_dir, exist_ok=True)\n",
    "\n",
    "# Load datasets (example, adjust as needed)\n",
    "dataset = load_dataset(\"FinGPT/fingpt-forecaster-dow30-202305-202405\", split=\"train\")\n",
    "\n",
    "# Load Llama-3.1-8B base model\n",
    "llama3_base = AutoModelForCausalLM.from_pretrained(\n",
    "    \"meta-llama/Llama-3.1-8B\",\n",
    "    trust_remote_code=True,\n",
    "    device_map=\"auto\",\n",
    "    cache_dir=cache_dir,\n",
    "    torch_dtype=torch.float16,\n",
    ")\n",
    "llama3_tokenizer = AutoTokenizer.from_pretrained(\"meta-llama/Llama-3.1-8B\", cache_dir=cache_dir)\n",
    "\n",
    "# Load DeepSeek base model\n",
    "deepseek_base = AutoModelForCausalLM.from_pretrained(\n",
    "    \"deepseek-ai/DeepSeek-R1-Distill-Llama-8B\",\n",
    "    trust_remote_code=True,\n",
    "    device_map=\"auto\",\n",
    "    cache_dir=cache_dir,\n",
    "    torch_dtype=torch.float16,\n",
    ")\n",
    "deepseek_tokenizer = AutoTokenizer.from_pretrained(\"deepseek-ai/DeepSeek-R1-Distill-Llama-8B\", cache_dir=cache_dir)\n",
    "\n",
    "# LoRA Fine-Tuning Configuration\n",
    "lora_config = LoraConfig(\n",
    "    r=8,\n",
    "    lora_alpha=16,\n",
    "    lora_dropout=0.1,\n",
    "    target_modules=[\"q_proj\", \"v_proj\"],\n",
    "    task_type=\"CAUSAL_LM\",\n",
    ")\n",
    "\n",
    "# Fine-tune Llama-3.1-8B\n",
    "llama3_model = get_peft_model(llama3_base, lora_config)\n",
    "# Add training loop here (simplified for example)\n",
    "llama3_model.save_pretrained(\"./finetuned_models/dow30-202305-202405-llama-3.1-8b_202602280001\")\n",
    "llama3_model = PeftModel.from_pretrained(llama3_base, \"./finetuned_models/dow30-202305-202405-llama-3.1-8b_202602280001\", cache_dir=cache_dir)\n",
    "llama3_model.eval()\n",
    "\n",
    "# Fine-tune DeepSeek-R1-Distill-Llama-8B\n",
    "deepseek_model = get_peft_model(deepseek_base, lora_config)\n",
    "# Add training loop here (simplified for example)\n",
    "deepseek_model.save_pretrained(\"./finetuned_models/dow30-202305-202405-DeepSeek-R1-Distill-llama-8B_202592020557\")\n",
    "deepseek_model = PeftModel.from_pretrained(deepseek_base, \"./finetuned_models/dow30-202305-202405-DeepSeek-R1-Distill-llama-8B_202592020557\", cache_dir=cache_dir)\n",
    "deepseek_model.eval()\n",
    "\n",
    "print(\"LoRA fine-tuning completed for both models.\")\n"
   ]
  }
 ],
 "metadata": {
  "language_info": {
   "name": "python"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 2
}